<a href="https://colab.research.google.com/github/Duncanswilson/llm-finetuning/blob/main/hellaswag_structured_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
import numpy as np

print(torch.cuda.is_available())

# Load model and tokenizer
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device('cuda:0')
model.to(device)

# Load HellaSwag validation dataset
dataset = load_dataset("hellaswag", split="validation[:1%]")

def compute_log_likelihood(model, tokenizer, context, ending):
    input_text = context + " " + ending
    inputs = tokenizer(input_text, return_tensors="pt")
    with torch.no_grad():
        inputs.to(device)
        outputs = model(**inputs)

    # Get logits and input IDs
    logits = outputs.logits
    input_ids = inputs["input_ids"]

    # Shift logits and input IDs for next-token prediction
    shift_logits = logits[:, :-1, :].contiguous()
    shift_labels = input_ids[:, 1:].contiguous()

    # Compute log-likelihood
    loss_fct = torch.nn.CrossEntropyLoss(reduction='sum')
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

    # Normalize by sequence length
    log_likelihood = -loss.item() / (input_ids.size(1) - 1)

    return log_likelihood

correct_predictions = 0
total_samples = len(dataset)
print(total_samples)
for sample in dataset:
    context = sample["ctx"]
    endings = sample["endings"]

    log_likelihoods = [compute_log_likelihood(model, tokenizer, context, ending) for ending in endings]
    predicted_index = np.argmax(log_likelihoods)
    correct_index = sample["label"]
    if predicted_index == int(correct_index):
        correct_predictions += 1

accuracy = correct_predictions / total_samples
print(f"Accuracy: {accuracy:.2f}")

True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


100
Accuracy: 0.28


In [4]:
!pip install outlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 44.0 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import outlines
import torch

# Load HellaSwag validation dataset
dataset = load_dataset("hellaswag", split="validation[:1%]")

# Create an Outlines generator
model = outlines.models.transformers("openai-community/gpt2")

def create_prompt(context, endings):
    prompt = f"""Given the following context and possible endings, choose the most likely continuation:

Context: {context}

Possible endings:
0. {endings[0]}
1. {endings[1]}
2. {endings[2]}
3. {endings[3]}

Choose the number of the most likely ending:"""
    return prompt

def get_model_choice(model, prompt):
    generator = outlines.generate.choice(model, ["0", "1", "2", "3"])
    answer = generator(prompt)
    return int(answer)

correct_predictions = 0
total_samples = len(dataset)
print(total_samples)

for sample in dataset:
    context = sample["ctx"]
    endings = sample["endings"]

    prompt = create_prompt(context, endings)
    predicted_index = get_model_choice(model, prompt)
    correct_index = sample["label"]

    if predicted_index == int(correct_index):
        correct_predictions += 1

accuracy = correct_predictions / total_samples
print(f"Accuracy: {accuracy:.2f}")

100
Accuracy: 0.25
